In [0]:
#!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import datetime

import numpy as np
import pandas as pd
%matplotlib
from matplotlib import pyplot as plt

In [0]:
path="data/COVID19_ZM_transformedTweets.csv"

spark = SparkSession \
    .builder \
    .appName("COVID-19 Twitter Sentiment Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
df = spark.read.csv(path, header=True, sep=",")
print(df.schema)

In [0]:
df = df.withColumn("dateTime", df["date"])
df = df.withColumn("date", df["date"].cast(DateType()))
df = df.withColumn("time", df["dateTime"].cast(TimestampType()))
df = df.withColumn("retweets", df["retweets"].cast(IntegerType()))

df.take(5)



In [0]:
df.select("username").dropDuplicates().sort("username").show()

In [0]:
df=df.withColumn("month", month(df.date))
df.take(5)

In [0]:
df=df.withColumn("day_of_month", dayofmonth(df.date))
df.take(5)

In [0]:
df=df.withColumn("year", year(df.date))
df.take(5)

In [0]:
df=df.withColumn("hour", hour(df.time))
df.take(5)

In [0]:
# TODO: get number of tweets per day
num_tweets_per_day=df.select("*").groupBy(df.date).count().orderBy(df.date)
num_tweets_per_day.sort("date",ascending=True).head(5)

In [0]:
# TODO: Check peoples feeling towards covid
retweets_per_day=df.select("retweets","date").groupBy("date").sum("retweets")
retweets_per_day.sort("date",ascending=True).head(100)

In [0]:
num_tweets_per_day.head(10)

In [0]:
# TODO: who tweets the most about COVID-19
freq_tweeters=df.select(df.username).groupBy(df.username).count()
freq_tweeters.sort("count",ascending=False).show()

In [0]:
# TODO: Check which hour people tweet the most
hours_with_most_tweets=df.select('*').groupBy(df.hour).count()
hours_with_most_tweets.sort("count",ascending=False).show()

In [0]:
# TODO: Check peoples feeling towards covid
retweets_per_hr=df.select("retweets","hour").groupBy("hour").sum("retweets")
retweets_per_hr.sort("hour",ascending=False).show()

In [0]:
# TODO: From the people with large following, based on the retwee
df.where(df.hour==23).select("retweets","hour").show()

In [0]:
mwebantu_tweets=df.filter(df.username=="Mwebantu")
mwebantu_tweets.show()